
# Projeto de Análise Estatística — `tips.csv` (Gorjetas em Restaurante)

**Objetivo:** cumprir os requisitos da *Lauda.pdf* criando um fluxo completo de análise:
- Exploração e limpeza dos dados
- Engenharia de variáveis
- Testes de hipóteses e Intervalos de Confiança
- Modelagem de Regressão Linear (prever gorjeta e analisar interdependências)
- Visualizações interativas (em notebook) e um **protótipo de dashboard** (opcional, via Streamlit)
- Recomendações práticas

> Arquivo base: `tips.csv` (fornecido).


In [ ]:

# %% [setup] Imports e Configurações
import pandas as pd
import numpy as np

# Testes e estatística
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Visualizações
import matplotlib.pyplot as plt  # Usaremos matplotlib conforme exigência do professor em algumas saídas
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['axes.grid'] = True

# Plotly para interatividade dentro do notebook (opcional)
import plotly.express as px

# Helper para IC de correlação (Fisher z)
def fisher_ci(r, n, alpha=0.05):
    z = np.arctanh(r)
    se = 1/np.sqrt(n-3)
    z_crit = stats.norm.ppf(1-alpha/2)
    lo = np.tanh(z - z_crit*se)
    hi = np.tanh(z + z_crit*se)
    return lo, hi

pd.set_option('display.float_format', lambda x: f'{x:,.4f}')
print('Bibliotecas carregadas.')


## 1. Carregamento do dataset

In [ ]:

# %% [data] Carregar a planilha tips.csv
path = '/mnt/data/tips.csv'  # ajustado para o ambiente de execução desta avaliação
df = pd.read_csv(path)

print(f'Linhas: {len(df)}, Colunas: {df.shape[1]}')
display(df.head())
display(df.tail())
display(df.dtypes)



### Dicionário de Dados (tips.csv)
- `total_bill`: valor total da conta (USD)
- `tip`: gorjeta (USD)
- `sex`: sexo do cliente pagante (Female/Male)
- `smoker`: cliente fumante? (Yes/No)
- `day`: dia da semana (Thur/Fri/Sat/Sun)
- `time`: período (Lunch/Dinner)
- `size`: tamanho da mesa (nº de pessoas)


## 2. Qualidade, Limpeza e Preparação dos Dados

In [ ]:

# Valores ausentes, duplicatas, estatísticas básicas
print('Valores ausentes por coluna:')
print(df.isna().sum())

print('\nDuplicatas:', df.duplicated().sum())

display(df.describe(include='all'))

# Regras mínimas de saneamento (consistência)
# - total_bill > 0
# - tip >= 0 e tip <= total_bill
# - size >= 1
mask_valid = (
    (df['total_bill'] > 0) &
    (df['tip'] >= 0) &
    (df['tip'] <= df['total_bill']) &
    (df['size'] >= 1)
)
df_clean = df.loc[mask_valid].copy()

# Tratamento de ausentes: numéricos por média, categóricos por moda (se existir ausentes)
num_cols = df_clean.select_dtypes(include=[np.number]).columns
df_clean[num_cols] = df_clean[num_cols].fillna(df_clean[num_cols].mean(numeric_only=True))

for col in df_clean.select_dtypes(include='object').columns:
    if df_clean[col].isna().any():
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode().iat[0])

print('Linhas após limpeza:', len(df_clean))
display(df_clean.describe(include='all'))


## 3. Engenharia de Variáveis

In [ ]:

# Novas variáveis úteis para análise e modelagem
df_clean['tip_pct'] = df_clean['tip'] / df_clean['total_bill']  # % da gorjeta
df_clean['is_weekend'] = df_clean['day'].isin(['Sat','Sun']).astype(int)
df_clean['is_dinner'] = (df_clean['time'] == 'Dinner').astype(int)
df_clean['is_smoker'] = (df_clean['smoker'] == 'Yes').astype(int)
df_clean['is_male'] = (df_clean['sex'] == 'Male').astype(int)

display(df_clean.head())


## 4. Análise Exploratória (EDA)

In [ ]:

# Estatísticas principais
desc = df_clean[['total_bill','tip','tip_pct','size']].describe()
display(desc)

# Correlação numérica
corr = df_clean[['total_bill','tip','tip_pct','size']].corr()
print('\nMatriz de correlação:')
display(corr)

# Gráficos rápidos com matplotlib (exigência professor)
fig, ax = plt.subplots()
ax.scatter(df_clean['total_bill'], df_clean['tip'])
ax.set_xlabel('Total da Conta')
ax.set_ylabel('Gorjeta')
ax.set_title('Dispersão: total_bill vs tip')
plt.show()

# Distribuição de tip_pct
fig, ax = plt.subplots()
ax.hist(df_clean['tip_pct'], bins=30)
ax.set_xlabel('tip_pct')
ax.set_ylabel('Frequência')
ax.set_title('Distribuição da porcentagem de gorjeta')
plt.show()

# Boxplots por categoria
fig, ax = plt.subplots()
df_clean.boxplot(column='tip', by='sex', ax=ax)
plt.suptitle('')
ax.set_title('Gorjeta por Sexo')
ax.set_xlabel('Sexo')
ax.set_ylabel('Gorjeta (USD)')
plt.show()

# Interatividade (opcional) - não obrigatório
try:
    fig_px = px.scatter(df_clean, x='total_bill', y='tip', color='sex', hover_data=['smoker','day','time','size'], trendline='ols')
    fig_px.update_layout(title='Plotly: total_bill vs tip (com trendline OLS)')
    fig_px.show()
except Exception as e:
    print('Plotly não disponível neste ambiente:', e)


## 5. Testes de Hipóteses e Intervalos de Confiança

In [ ]:

# Exemplos de hipóteses:
# H1: correlação entre total_bill e tip é > 0
r, p = stats.pearsonr(df_clean['total_bill'], df_clean['tip'])
n = df_clean.shape[0]
ci_lo, ci_hi = fisher_ci(r, n, alpha=0.05)

print(f'Correlação (total_bill, tip): r={r:.4f}, p-valor={p:.4f}, IC95%=({ci_lo:.4f}, {ci_hi:.4f})')

# H2: média de gorjeta difere entre sexos (t-test independente)
tip_male = df_clean.loc[df_clean['sex']=='Male', 'tip']
tip_fem  = df_clean.loc[df_clean['sex']=='Female', 'tip']
t_stat, p_val = stats.ttest_ind(tip_male, tip_fem, equal_var=False)  # Welch
print(f'\nT-test gorjeta: Male vs Female -> t={t_stat:.4f}, p={p_val:.4f}')

# IC95% para média de gorjeta por sexo
def mean_ci(series, alpha=0.05):
    xbar = series.mean()
    s = series.std(ddof=1)
    n = series.shape[0]
    tcrit = stats.t.ppf(1-alpha/2, df=n-1)
    se = s/np.sqrt(n)
    return xbar, xbar - tcrit*se, xbar + tcrit*se

m_male, lo_male, hi_male = mean_ci(tip_male)
m_fem, lo_fem, hi_fem = mean_ci(tip_fem)
print(f'Média Male: {m_male:.2f} (IC95%: {lo_male:.2f}, {hi_male:.2f})')
print(f'Média Female: {m_fem:.2f} (IC95%: {lo_fem:.2f}, {hi_fem:.2f})')

# H3: fumantes gastam mais? (Mann-Whitney em total_bill, caso distribuição não normal)
tb_smoke = df_clean.loc[df_clean['smoker']=='Yes','total_bill']
tb_nosmoke = df_clean.loc[df_clean['smoker']=='No','total_bill']
u_stat, p_val_u = stats.mannwhitneyu(tb_smoke, tb_nosmoke, alternative='two-sided')
print(f'\nMann-Whitney total_bill: smoker vs non-smoker -> U={u_stat:.0f}, p={p_val_u:.4f}')


## 6. Modelagem de Regressão Linear

In [ ]:

# Modelo 1: tip ~ total_bill
model1 = smf.ols('tip ~ total_bill', data=df_clean).fit()
print(model1.summary())

# Modelo 2: adicionando variáveis (sexo, fumante, tamanho, fim de semana, jantar)
model2 = smf.ols('tip ~ total_bill + is_male + is_smoker + size + is_weekend + is_dinner', data=df_clean).fit()
print(model2.summary())

# Modelo 3: prever proporção de gorjeta (bounded 0-1) — usamos OLS como proxy simples aqui
model3 = smf.ols('tip_pct ~ total_bill + is_male + is_smoker + size + is_weekend + is_dinner', data=df_clean).fit()
print(model3.summary())

# Avaliação simples via R^2 ajustado e RMSE
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

for i, m in enumerate([model1, model2, model3], start=1):
    if 'tip_pct' in m.model.endog_names:
        y_true = df_clean['tip_pct']
    else:
        y_true = df_clean['tip']
    y_pred = m.fittedvalues
    print(f'Modelo {i}: R2_aj={m.rsquared_adj:.4f} | RMSE={rmse(y_true, y_pred):.4f}')


## 7. Previsão de valores (exemplo)

In [ ]:

# Exemplo: prever gorjeta esperada dado um cenário
cenarios = pd.DataFrame({
    'total_bill': [20, 50, 80],
    'is_male': [1, 0, 1],
    'is_smoker': [0, 1, 0],
    'size': [2, 4, 5],
    'is_weekend': [0, 1, 1],
    'is_dinner': [1, 1, 0],
})
pred = model2.get_prediction(cenarios)
pred_df = pred.summary_frame(alpha=0.05)  # inclui IC95%
result = pd.concat([cenarios, pred_df], axis=1)
result.rename(columns={'mean':'tip_pred','mean_ci_lower':'IC95_inf','mean_ci_upper':'IC95_sup'}, inplace=True)
display(result)



## 8. Recomendações Práticas (baseadas nos achados)
- **Relação forte entre conta e gorjeta**: políticas de *suggested tip* podem usar faixas de `total_bill` para sugerir percentuais coerentes.
- **Tamanho da mesa importa**: grupos maiores tendem a dar gorjetas maiores em valor absoluto; ajustar serviço/recursos nas noites de fim de semana pode otimizar receita e satisfação.
- **Contexto (jantar/fim de semana)**: combinar escala de equipe com picos observados; priorizar garçons experientes em horários de maior ticket.
- **Dashboard**: acompanhar em tempo real `total_bill`, `tip`, `tip_pct`, e métricas por `day/time/smoker/sex` para decisões operacionais (escala, promoções, treinamento).



## 9. (Opcional) Dashboard interativo com Streamlit
O código abaixo grava um arquivo `app.py` com um protótipo de dashboard. Para rodar localmente:

```bash
pip install streamlit plotly pandas
streamlit run app.py
```


In [ ]:

# %% [dashboard] Gerar um protótipo de dashboard Streamlit (app.py)
app_code = r'''
import pandas as pd
import plotly.express as px
import streamlit as st

st.set_page_config(page_title="Dashboard - Tips", layout="wide")

@st.cache_data
def load_data():
    return pd.read_csv("tips.csv")

df = load_data()
df['tip_pct'] = df['tip']/df['total_bill']
df['is_weekend'] = df['day'].isin(['Sat','Sun']).astype(int)

st.title("Dashboard - Gorjetas (tips.csv)")

# Filtros
col1, col2, col3 = st.columns(3)
with col1:
    day_opt = st.multiselect("Dia(s)", sorted(df['day'].unique()), default=list(sorted(df['day'].unique())))
with col2:
    time_opt = st.multiselect("Período(s)", sorted(df['time'].unique()), default=list(sorted(df['time'].unique())))
with col3:
    smoker_opt = st.multiselect("Fumante", sorted(df['smoker'].unique()), default=list(sorted(df['smoker'].unique())))

mask = df['day'].isin(day_opt) & df['time'].isin(time_opt) & df['smoker'].isin(smoker_opt)
dff = df.loc[mask].copy()

# KPIs
c1, c2, c3, c4 = st.columns(4)
c1.metric("Ticket Médio", f"${dff['total_bill'].mean():.2f}")
c2.metric("Gorjeta Média", f"${dff['tip'].mean():.2f}")
c3.metric("Tip % Média", f"{(dff['tip_pct'].mean()*100):.1f}%")
c4.metric("Qtde Registros", f"{len(dff)}")

# Gráficos
fig1 = px.scatter(dff, x='total_bill', y='tip', color='sex', trendline='ols', hover_data=['day','time','smoker','size'])
fig1.update_layout(title='total_bill vs tip')

fig2 = px.box(dff, x='sex', y='tip', color='sex', title='Distribuição de Gorjeta por Sexo')
fig3 = px.histogram(dff, x='tip_pct', nbins=30, title='Distribuição de Tip %')

colA, colB = st.columns(2)
colA.plotly_chart(fig1, use_container_width=True)
colB.plotly_chart(fig2, use_container_width=True)

st.plotly_chart(fig3, use_container_width=True)

st.caption("Protótipo para avaliação acadêmica — Streamlit/Plotly.")
'''
with open('/mnt/data/app.py', 'w', encoding='utf-8') as f:
    f.write(app_code)

print("Arquivo app.py salvo em /mnt/data/app.py. Lembre-se de colocar o arquivo tips.csv na mesma pasta ao executar localmente.")



## 10. Referências e Notas
- Dataset clássico `tips` (Seaborn): documentação e uso didático em análises estatísticas.
- Conceitos: intervalos de confiança, testes t de Welch, correlação de Pearson, transformação de Fisher, regressão OLS (statsmodels).
- Ferramentas: pandas, numpy, scipy, statsmodels, matplotlib, plotly.

> Este notebook cobre os itens cobrados na **Lauda.pdf**: limpeza, EDA, testes de hipótese, ICs, regressão linear, previsão e dashboard.
